## Steps in Big Pictures
1. Initialize checker board
2. Get peg positions from server
3. update board with new peg positions
4. Calculate best move
5. Send move to server

## Initialize Checker Board
### Steps
1. Create a .json file with position.[maybe it would be hard coded but seems ok for this game, cause the coordinates are fixed]
2. Load the json 
3. Create the board as graph

In [61]:
# read json file to create a graph
import json
board = {}
with open('board.json', 'r') as f: #../
    data = json.load(f)
    for pos in data:
        #print(f"Type of pos: {type(pos)}")
        coordinate = list(pos.keys())[0]
        infos = pos[coordinate]
        board[coordinate] = infos
        
        #print(f"pos : {coordinate}")
        
#board

In [62]:
CONNECTED_COORDINATES = "conn_coors"
HEURISTIC = "heuristic"
TYPE = "type"
HOLD = "hold"

In [63]:
# test just simple move
def next_position_by_simple_move(cord):
    # check if adjacent positions has peg or not?
    # save the best heuristic position
    # cord="(-1,-1)"
    adj_pos = board[cord]['conn_coors']
    best_move = None
    for pos in adj_pos:
        pos_move = board[pos]['hold']
        if board[pos]['hold'] == None:
            if best_move == None:
                best_move = pos
            if board[best_move]['heuristic'] > board[pos]['heuristic']:
                best_move = pos
                
    # TODO: try to check if there 'S' shape is possible or not
                
    return best_move

pos = "3,-4"
print(f"next_position_by_simple_move for pos: {next_position_by_simple_move(pos)}")

next_position_by_simple_move for pos: 3,-3


In [5]:
jp_x, jp_y = int("3,-4".split(",")[0]), int("3,-4".split(",")[1])
#"3,-4" == ",".join([3,-4])
f"{3},{-4}" == "3,-4"
li = [("A", 2), ("S", 23), ("ZA", 0), ("YZ", -1)]
li.sort(key= lambda x: x[1], reverse= True)
li
import copy
reli = ["2,-3", "2,-2", "1,-1", "0,-1", "0,-2", "1,-3"]
celi = copy.copy(reli)
celi.remove("2,-3")
reli
sum([1,2,3,4,5])
d = {3: 6, 1: 2, 2: 4}
#sorted(d, )

In [64]:
# jp : "2,-3"
def is_jump_possible(jp: str, hp: str) -> bool:
    jumping_peg = jp.split(",")
    jp_x = int(jumping_peg[0])
    jp_y = int(jumping_peg[1])
    helper_peg = hp.split(",")
    hp_x = int(helper_peg[0])
    hp_y = int(helper_peg[1])
    
    
    # TODO: correctly define jumping peg
    
    x_diff = hp_x - jp_x
    y_diff = hp_y - jp_y
    
    print(f"Jumping : {jp}, helper: {hp}")
    hoping_pos = f"{hp_x + x_diff},{hp_y + y_diff}"
    # print(hoping_pos)
    # if hoping position is free to move then return position or return none
    if hoping_pos in board and board[hoping_pos]['hold'] == None:
        return hoping_pos
    else:
        return None
    
is_jump_possible("3,-4", "2,-3")

Jumping : 3,-4, helper: 2,-3


'1,-2'

In [66]:
import copy
def moving_by_hop_chain(start_cord, curr_cord, pre_hp_cord):
    if is_stop_for_hop_chain(start_cord, curr_cord, pre_hp_cord):
        return [curr_cord]
    
    possible_moves = []
    
    adj_pos = copy.copy(board[curr_cord]['conn_coors'])
    adj_pos.remove(pre_hp_cord)
    
    for pos in adj_pos:
        possible_moves.append([curr_cord], [board[curr_cord][HEURISTIC]])
        if board[pos]['hold'] != None:
            jump_pos = is_jump_possible(curr_cord, pos)
            if jump_pos != None:
                moving_pos_for_jump = moving_by_hop_chain(start_cord, jump_pos, pos)
                
                path = [start_cord] + moving_pos_for_jump[0]
                board_val = board[moving_pos_for_jump[0][-1]]
                possible_moves.append([path, board_val['heuristic']])
    possible_moves.sort(key= lambda x: x[1]) # sort based on heuristic value
    print(f"POssible moves: {possible_moves}")
    return possible_moves[0] # first tuple -> first value

moving_by_hop_chain("3,-4", "1,-2", "2,-3")
#moving_by_hop_chain("2,-1", "0,-1", "1,-1")

['1,-2']

In [65]:
def is_stop_for_hop_chain(start_cord, moving_cord, pre_hp_cord):
    # if there are no position for moving by hop chain
    adj_pos = board[moving_cord]['conn_coors'] #possible delete pre_hp_cord
    hop_check_list = []
    for pos in adj_pos:
        # TODO: check if swap is possible or not
        if board[pos]['hold'] != None and pos != pre_hp_cord:
            hop_pos = is_jump_possible(moving_cord, pos)
            if hop_pos != None and hop_pos != start_cord:
                return False
    return True
is_stop_for_hop_chain("3,-4", "-1,2", "0,1")

True

In [8]:
peg_curr_pos = ["3,-4", "2,-3", "0,-1", "0,2", "-1,3", "-2,5"]

In [9]:
def cal_game_adv_score(pegs: [str]) -> float:
    return sum([board[peg]['heuristic'] for peg in pegs])

In [18]:
def get_next_move():
    # TODO: remove the peg which is already in home and no way to move
    # TODO: first get all current positions of our pegs
    curr_game_score = cal_game_adv_score(peg_curr_pos)
    # TODO: find out which move we are going to make, [simple, simple hop, hop chain]
    peg_score = {}
    for peg in peg_curr_pos:
        next_move = next_move_for_one_peg(peg)
        game_adv = board[peg][HEURISTIC] - board[next_move][HEURISTIC]
        peg_score[peg] = (peg, next_move, curr_game_score - game_adv)
    # TODO: calculate board advanced score based on pegs
    # TODO: find the peg we should move, which peg minimize our value and maximize other score
    min_value = None
    val = None
    for key, value in peg_score.items():
        if min_value == None:
            min_value = value[2]
            val = value
        elif min_value and value[2] < min_value:
            min_value = value[2]
            val = value
    return val
peg, next_peg, score = get_next_move()
print(f"Select peg : {peg} for miving to : {next_peg}, cause score is minimum: {score}")

Jumping : 3,-4, helper: 2,-3
Jumping : 1,-2, helper: 0,-1
Jumping : 1,-2, helper: 0,-1
Jumping : -1,0, helper: 0,0
Jumping : -1,0, helper: 0,0
Jumping : 1,0, helper: 0,1
Jumping : 1,0, helper: 0,1
Jumping : -1,2, helper: 0,2
Jumping : -1,2, helper: 0,2
Jumping : -1,2, helper: -1,3
Jumping : -1,4, helper: -2,5
Jumping : -1,4, helper: -2,5
Jumping : -1,0, helper: -2,0
Jumping : -3,0, helper: -4,1
Jumping : -3,0, helper: -3,-1
Jumping : -1,0, helper: -1,-1
Jumping : 2,-3, helper: 3,-4
Jumping : 0,-1, helper: 0,0
Jumping : 0,-1, helper: -1,-1
Jumping : -2,-1, helper: -1,-2
Jumping : -2,-1, helper: -1,-2
Jumping : 0,-3, helper: -1,-3
Jumping : -2,-1, helper: -2,0
Jumping : -2,-1, helper: -3,-1
Jumping : 0,2, helper: -1,3
Jumping : -2,4, helper: -2,5
Jumping : -2,4, helper: -1,-3
Jumping : 0,2, helper: 0,1
Jumping : -1,3, helper: 0,2
Jumping : 1,1, helper: 0,1
Jumping : 1,1, helper: 0,1
Jumping : -1,1, helper: 0,0
Jumping : -1,1, helper: 0,0
Jumping : 1,-1, helper: 0,-1
Select peg : 3,-4 for

In [17]:
def next_move_for_one_peg(peg_pos: str) -> str:
    adj_pos = board[peg_pos][CONNECTED_COORDINATES]
    possible_moves = {}
    for a_pos in adj_pos:
        # jump possible
        if board[a_pos][HOLD] != None:
            hop_pos = is_jump_possible(peg_pos, a_pos)
            if hop_pos != None:
                hop_chain_pos = moving_by_hop_chain(peg_pos ,hop_pos, a_pos)
                possible_moves[hop_chain_pos] = [board[hop_chain_pos][HEURISTIC], 'H']
        # jump not possible, check for sigle move
        else:
            if board[a_pos][HOLD] == None:
                possible_moves[a_pos] = [board[a_pos][HEURISTIC], 'S']
    # sort the dict based on for finding optimize move
    opt_move = None
    for key, value in possible_moves.items():
        if opt_move == None:
            opt_move = key
        else:
            if possible_moves[opt_move][0] > possible_moves[key][0]:
                opt_move = key
                
    # TODO: test if move is simple move then try to create shape of 'S' if possible
    # then return opt_move
    return opt_move

peg = "-1,3"
print(f"Move for peg : {peg} is -> {next_move_for_one_peg(peg)}")

Jumping : -1,3, helper: 0,2
Jumping : 1,1, helper: 0,1
Jumping : 1,1, helper: 0,1
Jumping : -1,1, helper: 0,0
Jumping : -1,1, helper: 0,0
Jumping : 1,-1, helper: 0,-1
Move for peg : -1,3 is -> -1,4


In [ ]:
# test S shaped conversion
# test simple hop move
# test hop chain move
# calculate next moving probable coordinates
# test recursive case for one coordinate move for one peg
# return the max value for 

In [ ]:
def get_next_hop(pos: (int, int)) -> (int, int):
    # adjacent nodes for this position
    # if one position has value then find the position behind it
    # if the hoping position is empty then add it to list
    # for 3,-4 ad_list = 3,-5; 2,-4; 2,-3; 3,-3
    # 2,-3 has peg, x_diff = 2 - 3, y_diff = -3 + 4 then behind_of_pos = 2 + x_diff, -3 + y_diff
    

In [ ]:
def next_move_by_jump(pos: )

In [ ]:
# TODO: calculate the moves for all pegs and find the best one
# consider other's move with the peg by itself
# min max algo

In [11]:
string = "-2 ,5; -1 ,3;0 , -1;0 ,2;2 , -3;3 , -4:-3 , -2; -3 , -1; -1 , -2;0 ,0;0 ,1; -1 , -1:-5 ,2; -4 ,1; -4 ,3; -2 , -3; -2 ,0; -1 , -3"
positions = string.split(':')
a_pegs =  [remove_space(pos) for pos in positions[0].split(';')]
a_pegs

['-2,5', '-1,3', '0,-1', '0,2', '2,-3', '3,-4']

In [34]:
"-2 ,5; -1 ,3;0 , -1;0 ,2;2 , -3;3 , -4".split(';')[0].split(',')
['-2,5', '-1,3', '0,-1', '0,2', '2,-3', '3,-4'].index('0,-1')

2

In [10]:
def remove_space(s: str) -> str:
    new_s = ""
    for c in s:
        if not c.isspace():
            new_s += c
    return new_s
remove_space('-2 ,5')

'-2,5'

In [13]:
a = "A"
f"{a}_home" == "A_home"

True

In [44]:
peg_turn = {"A": 0, "B": 1, "C": 2}
turn_peg = {0: "A", 1: "B", 2: "C"}
board["-3,-3"][HOLD] = None
print(board["-3,-3"])
def swapable(hoping_pos, turn):
    # check if swap is possible or not
    peg = board[hoping_pos]
    peg_n = turn_peg[turn]
    peg_hold = peg[HOLD]
    home_peg = f"{peg_n}_home"
    print(f"{home_peg}, {peg[TYPE]}, {peg_hold}, {peg_n}")
    if home_peg == peg[TYPE] and peg_hold != None and peg_hold != peg_n:
        return True
    else:
        return False
    
swapable("-3,-3", 2)

{'conn_coors': ['-2,-3', '-3,-2'], 'heuristic': [9, 0, 9], 'type': 'B_home', 'hold': None}
C_home, B_home, None, C


False

In [32]:
def is_jump_possible(jp: str, hp: str, turn):
    jumping_peg = jp.split(",")
    jp_x = int(jumping_peg[0])
    jp_y = int(jumping_peg[1])
    helper_peg = hp.split(",")
    hp_x = int(helper_peg[0])
    hp_y = int(helper_peg[1])

    # TODO: correctly define jumping peg

    x_diff = hp_x - jp_x
    y_diff = hp_y - jp_y

    # print(f"Jumping : {jp}, helper: {hp}")
    hoping_pos = f"{hp_x + x_diff},{hp_y + y_diff}"
    # print(hoping_pos)
    # if hoping position is free to move then return position or return none
    if hoping_pos in board:
        hoping_pos_peg = board[hoping_pos][HOLD]
        if hoping_pos_peg == None:
            return hoping_pos
        else:
            if swapable(hoping_pos, turn):
                # swap two peg
                temp = hoping_pos_peg
                board[hoping_pos][HOLD] = None
                board[jp][HOLD] = temp
                return hoping_pos
            return None

board["-2,5"][HOLD] = "A"        
is_jump_possible("-1,4", "-2,5", 0)

A_home, A_home, B, A


'-3,6'

In [37]:
li = [("A", 2), ("S", 23), ("ZA", 0), ("YZ", -1)]
li.sort(key= lambda x: x[1], reverse= False)
li

[('YZ', -1), ('ZA', 0), ('A', 2), ('S', 23)]

In [ ]:
{
                "A" : ["3,-6", "2,-5", "3,-5", "1,-4", "2,-4", "3,-4"],
                "B" : ["3,3", "2,3", "3,2", "1,3", "2,2", "3,1"],
                "C" : ["-6,3", "-5,3", "-5,2", "-4,1", "-4,2", "-4,3"]
            }

{
                "A" : ["3,-4", "2,-3", "0,-1", "0,2", "-1,3", "-2,5"],
                "B" : ["-3,-2", "-3,-1", "-1,-1", "0,0", "-1,-2", "0,1"],
                "C" : ["-5,2", "-4,1", "-4,3", "-2,0", "-2,-3", "-1,-3"]
            }

In [78]:
import copy
pos = {         "A" : ["3,-4", "2,-3", "0,-1", "0,2", "-1,3", "-2,5"],
                "B" : ["-3,-2", "-3,-1", "-1,-1", "0,0", "-1,-2", "0,1"],
                "C" : ["-5,2", "-4,1", "-4,3", "-2,0", "-2,-3", "-1,-3"]
            }
c_pos = copy.deepcopy(pos)
c_pos["A"][0] = "-3,6"
pos, c_pos

({'A': ['3,-4', '2,-3', '0,-1', '0,2', '-1,3', '-2,5'],
  'B': ['-3,-2', '-3,-1', '-1,-1', '0,0', '-1,-2', '0,1'],
  'C': ['-5,2', '-4,1', '-4,3', '-2,0', '-2,-3', '-1,-3']},
 {'A': ['-3,6', '2,-3', '0,-1', '0,2', '-1,3', '-2,5'],
  'B': ['-3,-2', '-3,-1', '-1,-1', '0,0', '-1,-2', '0,1'],
  'C': ['-5,2', '-4,1', '-4,3', '-2,0', '-2,-3', '-1,-3']})

In [72]:
board
def reset_board(board):
    for key, value in board.items():
        value['hold'] = None
    return board
board = reset_board(board)
board
def update_pegs_info(board, peg_pos):
    # reset board
    res_board = reset_board(board)
    # update board
    for key, value in peg_pos.items():
        for val in value:
            res_board[val]['hold'] = key
    return res_board
#board = update_pegs_info(board, pos)
#board

In [79]:
";".join((0, ['3,-4', '1,-2', '-1,0', '1,0', '-1,2', '-1,4', '-3,6'], 'H')[1])

'3,-4;1,-2;-1,0;1,0;-1,2;-1,4;-3,6'

In [83]:
def is_initial(positions: [str], turn) -> bool:
    start_peg = "A_start"
    # experimental if number of list is greater or equal to 3, we treat it as a initial game
    return len([pos for pos in positions if board[pos][HOLD] == "A" and board[pos][TYPE]==start_peg]) >= 3
pos = ['1,-4', '2,-4', '3,-4', '2,-5', '3,-5', '3,-6']
is_initial(pos, 0)

False

In [84]:
['1,-4', '2,-4', '3,-4', '2,-5', '3,-5', '3,-6'].index('3,-4')

2

In [85]:
a, b = 2, 3

In [87]:
"3,-4"=='3,-4'

True